# Phase 9: Clinician Web Interface (Flask Deployment Demo)

## 🎯 Overview

This phase creates a **production-ready web application** that allows healthcare professionals to input patient data and receive instant heart disease risk predictions. The Flask app runs directly from this Jupyter notebook, making it easy to demo and deploy.

## 📋 What This Phase Includes

### 1. **Web-Based Clinician Interface**
   - Simple, intuitive form for entering patient data
   - All 11 required patient features with proper input types
   - Real-time predictions with risk assessment

### 2. **Complete Prediction Pipeline**
   - Loads trained model artifacts (model, scaler, encoders)
   - Applies same preprocessing as training phase:
     - Label encoding for binary features (Sex, ExerciseAngina)
     - One-hot encoding for categorical features
     - Feature engineering (HR_Age_Ratio, Cholesterol_High, Age_Group)
     - Feature scaling
   - Returns comprehensive prediction results

### 3. **Risk Assessment Output**
   - **Disease Probability**: Percentage chance of heart disease (0-100%)
   - **Risk Level**: LOW RISK (0-30%), MODERATE RISK (30-70%), HIGH RISK (70-100%)
   - **Prediction**: Binary classification (Has Disease / No Disease)
   - **Confidence**: Model's confidence in the prediction

## 🚀 How to Use

1. **Run the setup cell** (Cell 1) to load all model artifacts
2. **Run the Flask app cell** (Cell 2) to start the web server
3. **Click the link** that appears (http://localhost:5000) to open the clinician interface
4. **Enter patient data** in the web form and click "Predict"
5. **View results** showing risk assessment and recommendations

## ⚠️ Important Notes

- The Flask server runs in the notebook and will block until stopped
- To stop the server, interrupt the kernel or close the notebook
- For production deployment, export the Flask app to a standalone Python file
- Ensure all model artifacts are in the `models/` directory before running

## 🔧 Technical Details

- **Framework**: Flask (lightweight Python web framework)
- **Model**: Trained Random Forest/XGBoost classifier from Phase 7
- **Preprocessing**: Matches Phase 4 preprocessing pipeline exactly
- **Port**: Default 5000 (change if needed)
- **Debug Mode**: Disabled for production-like behavior


In [1]:
import os
import json
import pickle
import pandas as pd
import numpy as np
from flask import Flask, request, render_template_string

print("="*80)
print("✓ PHASE 9: CLINICIAN WEB INTERFACE SETUP")
print("="*80)

# --- Load artifacts ---
feature_info_path = 'models/feature_info.json'
alt_feature_info_path = 'models/models/feature_info.json'

if os.path.exists(feature_info_path):
    with open(feature_info_path, 'r') as f:
        feature_info = json.load(f)
else:
    feature_info = {
        'numerical_features': ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak', 'HR_Age_Ratio'],
        'categorical_features': ['Sex', 'ExerciseAngina', 'ChestPainType', 'RestingECG', 'ST_Slope'],
        'feature_order': []
    }

# fallback to packaged feature order if missing
if not feature_info.get('feature_order') and os.path.exists(alt_feature_info_path):
    with open(alt_feature_info_path, 'r') as f:
        feature_info = json.load(f)
    # Save it for future use
    with open(feature_info_path, 'w') as f:
        json.dump(feature_info, f)

with open('models/best_model_tuned.pkl', 'rb') as f:
    model = pickle.load(f)
with open('models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('models/encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

print("\n✓ Model artifacts loaded successfully!")
print(f"  - Model: {type(model).__name__}")
print(f"  - Features expected: {len(feature_info.get('feature_order', []))}")
print("\n" + "-"*80)


✓ PHASE 9: CLINICIAN WEB INTERFACE SETUP


FileNotFoundError: [Errno 2] No such file or directory: 'models/scaler.pkl'

In [ ]:
# --- Prediction helper function (matches Phase 8 with all fixes) ---
def predict_patient(patient_dict: dict):
    """Predict heart disease for new patient with complete preprocessing pipeline"""
    try:
        patient_df = pd.DataFrame([patient_dict])

        # Apply label encoders for binary features
        for col in ['Sex', 'ExerciseAngina']:
            if col in patient_df and col in encoders:
                try:
                    patient_df[col] = encoders[col].transform(patient_df[col])
                except Exception:
                    pass

        # Recreate engineered features used at training time
        if 'Age' in patient_df and 'MaxHR' in patient_df:
            patient_df['HR_Age_Ratio'] = patient_df['MaxHR'] / (patient_df['Age'] + 1)
        if 'Cholesterol' in patient_df:
            patient_df['Cholesterol_High'] = (patient_df['Cholesterol'] > 200).astype(int)
        if 'Age' in patient_df:
            try:
                patient_df['Age_Group'] = pd.cut(patient_df['Age'], bins=[0, 30, 40, 50, 60, 100], labels=[0, 1, 2, 3, 4]).astype(int)
            except Exception:
                # fallback: simple bucket
                patient_df['Age_Group'] = pd.Series(patient_df['Age']).apply(
                    lambda x: 0 if x < 30 else 1 if x < 40 else 2 if x < 50 else 3 if x < 60 else 4
                ).values

        # One-hot encode multiclass categoricals (keep all categories)
        for col in feature_info.get('categorical_features', []):
            if col in patient_df and col not in ['Sex', 'ExerciseAngina']:
                dummies = pd.get_dummies(patient_df[col], prefix=col, drop_first=False)
                patient_df = pd.concat([patient_df.drop(columns=[col]), dummies], axis=1)

        # Ensure numeric columns exist before scaling
        num_cols = feature_info.get('numerical_features', [])
        for c in num_cols:
            if c not in patient_df:
                patient_df[c] = 0

        # Scale numeric columns
        if num_cols:
            patient_df[num_cols] = scaler.transform(patient_df[num_cols])

        # Build final feature vector matching model's expected columns
        model_cols = list(getattr(model, 'feature_names_in_', []))
        feature_order = feature_info.get('feature_order', [])
        target_cols = model_cols or feature_order or list(patient_df.columns)

        # Initialize final dataframe with zeros and correct columns
        final_row = {c: 0 for c in target_cols}

        # Fill known columns from patient_df
        for col in patient_df.columns:
            if col in final_row:
                val = patient_df.iloc[0][col]
                if hasattr(val, 'item'):
                    try:
                        val = val.item()
                    except Exception:
                        pass
                final_row[col] = val

        final_df = pd.DataFrame([final_row], columns=target_cols).replace({np.nan: 0})

        # Ensure numeric dtypes
        for c in final_df.columns:
            try:
                final_df[c] = pd.to_numeric(final_df[c])
            except Exception:
                pass

        # Predict
        disease_prob = model.predict_proba(final_df)[0, 1]
        prediction = model.predict(final_df)[0]
        risk = "HIGH RISK" if disease_prob > 0.7 else ("MODERATE RISK" if disease_prob > 0.4 else "LOW RISK")

        return {
            'disease_probability': round(float(disease_prob), 4),
            'risk_level': risk,
            'prediction': 'Has Disease' if int(prediction) == 1 else 'No Disease',
            'confidence': round(max(disease_prob, 1 - disease_prob), 4)
        }
    except Exception as e:
        return {'error': str(e)}

print("✓ Prediction function ready!")


In [ ]:
# --- Flask Web Application ---
app = Flask(__name__)

# HTML Template for the clinician interface
FORM_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>Heart Disease Risk Assessment</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 50px auto;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .container {
            background: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        h1 {
            color: #2c3e50;
            text-align: center;
            margin-bottom: 30px;
        }
        .form-group {
            margin-bottom: 15px;
        }
        label {
            display: block;
            margin-bottom: 5px;
            color: #34495e;
            font-weight: bold;
        }
        input[type="number"], select {
            width: 100%;
            padding: 8px;
            border: 1px solid #ddd;
            border-radius: 4px;
            font-size: 14px;
        }
        button {
            background-color: #3498db;
            color: white;
            padding: 12px 30px;
            border: none;
            border-radius: 5px;
            font-size: 16px;
            cursor: pointer;
            width: 100%;
            margin-top: 20px;
        }
        button:hover {
            background-color: #2980b9;
        }
        .result {
            margin-top: 30px;
            padding: 20px;
            border-radius: 5px;
        }
        .result.high-risk {
            background-color: #fee;
            border-left: 4px solid #e74c3c;
        }
        .result.moderate-risk {
            background-color: #fff9e6;
            border-left: 4px solid #f39c12;
        }
        .result.low-risk {
            background-color: #e8f5e9;
            border-left: 4px solid #27ae60;
        }
        .result h2 {
            margin-top: 0;
        }
        .back-link {
            display: block;
            text-align: center;
            margin-top: 20px;
            color: #3498db;
            text-decoration: none;
        }
        .back-link:hover {
            text-decoration: underline;
        }
        .error {
            background-color: #fee;
            color: #c33;
            padding: 15px;
            border-radius: 5px;
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🏥 Heart Disease Risk Assessment Tool</h1>
        <p style="text-align: center; color: #7f8c8d;">Enter patient information below to get an instant risk assessment</p>
        
        <form action="/predict" method="post">
            <div class="form-group">
                <label>Age (years):</label>
                <input name="Age" type="number" value="55" min="0" max="120" required>
            </div>
            
            <div class="form-group">
                <label>Sex:</label>
                <select name="Sex" required>
                    <option value="M">Male</option>
                    <option value="F">Female</option>
                </select>
            </div>
            
            <div class="form-group">
                <label>Chest Pain Type:</label>
                <select name="ChestPainType" required>
                    <option value="ATA">ATA - Atypical Angina</option>
                    <option value="NAP">NAP - Non-Anginal Pain</option>
                    <option value="TA">TA - Typical Angina</option>
                    <option value="ASY">ASY - Asymptomatic</option>
                </select>
            </div>
            
            <div class="form-group">
                <label>Resting Blood Pressure (mm Hg):</label>
                <input name="RestingBP" type="number" value="140" min="0" max="300" required>
            </div>
            
            <div class="form-group">
                <label>Cholesterol (mg/dl):</label>
                <input name="Cholesterol" type="number" value="260" min="0" max="700" required>
            </div>
            
            <div class="form-group">
                <label>Fasting Blood Sugar:</label>
                <select name="FastingBS" required>
                    <option value="0">≤ 120 mg/dl (Normal)</option>
                    <option value="1">> 120 mg/dl (Elevated)</option>
                </select>
            </div>
            
            <div class="form-group">
                <label>Resting ECG:</label>
                <select name="RestingECG" required>
                    <option value="Normal">Normal</option>
                    <option value="ST">ST-T Wave Abnormality</option>
                    <option value="LVH">Left Ventricular Hypertrophy</option>
                </select>
            </div>
            
            <div class="form-group">
                <label>Max Heart Rate (bpm):</label>
                <input name="MaxHR" type="number" value="150" min="0" max="250" required>
            </div>
            
            <div class="form-group">
                <label>Exercise Angina:</label>
                <select name="ExerciseAngina" required>
                    <option value="N">No</option>
                    <option value="Y">Yes</option>
                </select>
            </div>
            
            <div class="form-group">
                <label>Oldpeak (ST depression):</label>
                <input name="Oldpeak" type="number" step="0.1" value="1.0" min="0" max="10" required>
            </div>
            
            <div class="form-group">
                <label>ST Slope:</label>
                <select name="ST_Slope" required>
                    <option value="Up">Up</option>
                    <option value="Flat">Flat</option>
                    <option value="Down">Down</option>
                </select>
            </div>
            
            <button type="submit">🔍 Assess Risk</button>
        </form>
    </div>
</body>
</html>
"""

@app.route("/")
def index():
    return FORM_HTML

@app.route("/predict", methods=["POST"])
def predict():
    # Extract form values with safe defaults
    try:
        payload = {
            'Age': float(request.form.get('Age', 0)),
            'Sex': request.form.get('Sex', 'M'),
            'ChestPainType': request.form.get('ChestPainType', 'ATA'),
            'RestingBP': float(request.form.get('RestingBP', 0)),
            'Cholesterol': float(request.form.get('Cholesterol', 0)),
            'FastingBS': int(request.form.get('FastingBS', 0)),
            'RestingECG': request.form.get('RestingECG', 'Normal'),
            'MaxHR': float(request.form.get('MaxHR', 0)),
            'ExerciseAngina': request.form.get('ExerciseAngina', 'N'),
            'Oldpeak': float(request.form.get('Oldpeak', 0)),
            'ST_Slope': request.form.get('ST_Slope', 'Up'),
        }

        result = predict_patient(payload)

        if 'error' in result:
            return f"""
            <div class="container">
                <div class="error">
                    <h2>Error</h2>
                    <p>{result['error']}</p>
                    <a href='/' class="back-link">← Back to Form</a>
                </div>
            </div>
            """

        # Determine risk class for styling
        risk_class = result['risk_level'].lower().replace(' ', '-')
        prob_pct = result['disease_probability'] * 100
        conf_pct = result['confidence'] * 100

        return f"""
        <div class="container">
            <h1>🏥 Heart Disease Risk Assessment</h1>
            <div class="result {risk_class}">
                <h2>Prediction Results</h2>
                <p><strong>Prediction:</strong> {result['prediction']}</p>
                <p><strong>Risk Level:</strong> {result['risk_level']}</p>
                <p><strong>Disease Probability:</strong> {prob_pct:.1f}%</p>
                <p><strong>Model Confidence:</strong> {conf_pct:.1f}%</p>
            </div>
            <a href='/' class="back-link">← Assess Another Patient</a>
        </div>
        """
    except Exception as e:
        return f"""
        <div class="container">
            <div class="error">
                <h2>Error</h2>
                <p>An error occurred: {str(e)}</p>
                <a href='/' class="back-link">← Back to Form</a>
            </div>
        </div>
        """

print("\n" + "="*80)
print("✓ Flask app configured!")
print("\n📌 To start the web server, run the next cell.")
print("="*80)


In [ ]:
# Optional: Test prediction function before starting server
print("\n" + "="*80)
print("🧪 TESTING PREDICTION FUNCTION")
print("="*80)

test_patient = {
    'Age': 55, 'Sex': 'M', 'ChestPainType': 'ATA',
    'RestingBP': 140, 'Cholesterol': 260, 'FastingBS': 1,
    'RestingECG': 'Normal', 'MaxHR': 145, 'ExerciseAngina': 'N',
    'Oldpeak': 2.0, 'ST_Slope': 'Flat'
}

test_result = predict_patient(test_patient)
if 'error' in test_result:
    print(f"❌ Test failed: {test_result['error']}")
    print("\n⚠️ Please check your model artifacts before starting the server.")
else:
    print(f"✓ Test successful!")
    print(f"  Prediction: {test_result['prediction']}")
    print(f"  Risk Level: {test_result['risk_level']}")
    print(f"  Probability: {test_result['disease_probability']*100:.1f}%")
    print("\n✓ Ready to start web server!")

print("="*80)


In [ ]:
# Start the Flask web server
# This will block until you interrupt the kernel
# Open http://localhost:5000 in your browser to use the clinician interface

print("\n" + "="*80)
print("🚀 STARTING FLASK WEB SERVER")
print("="*80)
print("\n📌 The web interface is now available at:")
print("   👉 http://localhost:5000")
print("\n💡 To stop the server, interrupt the kernel (Kernel → Interrupt)")
print("="*80 + "\n")

app.run(host='127.0.0.1', port=5000, debug=False)


## 📊 Understanding the Results

### Risk Levels Explained

- **LOW RISK (0-30% probability)**: Patient shows minimal indicators of heart disease. Routine monitoring recommended.

- **MODERATE RISK (30-70% probability)**: Some risk factors present. Further diagnostic tests may be warranted. Lifestyle modifications recommended.

- **HIGH RISK (70-100% probability)**: Strong indicators of heart disease. Immediate clinical evaluation and comprehensive cardiac assessment strongly recommended.

### Model Outputs

- **Disease Probability**: The model's estimate of the likelihood that the patient has heart disease (0-100%)
- **Prediction**: Binary classification based on the probability threshold
- **Confidence**: How certain the model is about its prediction (higher is better)
- **Risk Level**: Categorized assessment for quick clinical interpretation

## 🔍 Feature Descriptions

- **Age**: Patient's age in years
- **Sex**: Biological sex (M/F)
- **ChestPainType**: Type of chest pain experienced
  - ATA: Atypical Angina
  - NAP: Non-Anginal Pain
  - TA: Typical Angina
  - ASY: Asymptomatic
- **RestingBP**: Resting blood pressure (mm Hg)
- **Cholesterol**: Serum cholesterol level (mg/dl)
- **FastingBS**: Fasting blood sugar (>120 mg/dl = 1, else 0)
- **RestingECG**: Resting electrocardiographic results
- **MaxHR**: Maximum heart rate achieved during exercise
- **ExerciseAngina**: Exercise-induced angina (Y/N)
- **Oldpeak**: ST depression induced by exercise relative to rest
- **ST_Slope**: Slope of the peak exercise ST segment

## ⚠️ Clinical Disclaimer

**This tool is for demonstration and educational purposes only.** 

- The predictions are based on machine learning models trained on historical data
- Results should **NOT** replace professional medical judgment
- Always consult qualified healthcare professionals for actual patient care decisions
- Model performance may vary with different patient populations
- Use at your own discretion and responsibility

## 🛠️ Troubleshooting

**Server won't start?**
- Check if port 5000 is already in use
- Ensure Flask is installed: `pip install flask`
- Verify all model artifacts exist in `models/` directory

**Prediction errors?**
- Ensure all form fields are filled correctly
- Check that model files are not corrupted
- Verify feature_info.json contains correct feature order

**Can't access the web interface?**
- Make sure the Flask server cell is running (not interrupted)
- Try accessing http://127.0.0.1:5000 instead
- Check firewall settings if accessing from another machine
